### <center>Amplicación de análisis de estructuras. Máster Universitario en Ingeniería de Caminos, Canales y Puertos. Universidad de Granada. Curso 2023-2024<center>

# <center>Software para la resolución numérica del Problema de Placas Rectangulares mediante el Método de Navier.<center>

#### <center>Antonio Gomez. Marzo 2024<center>

Este script de Jupyter lab contiene una serie de algoritmos escritos en lenguaje Python para calcular y evaluar la flecha w(x,y) y los giros theta(x,y) mediante el método de Navier para poder resolver la tarea adjunta.

Los casos disponibles de resolución son los de:
- Carga uniforme distribuida de valor p.
- Carga puntual de valor 1, es decir, la Función de Green del problema placa.
- Carga puntual de valor F.
- Momento flector Mx(y) distribuido en línea x = xi
- Momento flector My(x) distribuido en línea y = eta

Nota: use únicamente los casos que le sean relevantes para resolver la tarea.

Adicionalmente, este script también permite el cálculo de:
- Los términos Mm del desarrollo en serie de un momento flector distribuido Mx(y) fruto de empotrar los bordes x=0 y x=a.
- Los términos Mn del desarrollo en serie de un momento flector distribuido My(x) fruto de empotrar los bordes y=0 e y=b.

Complete con datos aquellas variables sin definir (verá que tienen puntos suspensivos). Tendrá que superponer flechas debidas a diferentes estados de carga para resolver las preguntas de la tarea (superponer flechas significa sumar matrices de flechas w_xy debidas a los diferentes estados de carga involucrados en tu caso).

**Índice:**
1. Importamos las librerías necesarias
2. Datos de entrada
3. Flecha y giros generados por una carga uniforme distribuida w<sub>p</sub>(x,y)
4. Flecha generada por una carga puntual de valor 1 aplicada en un punto ($\xi$,$\eta$), es decir, la Función de Green del problema placa K(x,y;$\xi$,$\eta$)
5. Cálculo de fuerza reactiva F de un pilar biarticulado bajo el punto central de la placa.
6. Flecha generada por una carga puntual de valor F aplicada en un punto ($\xi$,$\eta$).
7. Cálculo de momento flector My(x) en empotramientos de problema simétrico en $\eta_1$ = 0 y $\eta_2$ = b.
8. Cálculo de momento flector Mx(y) en empotramientos de problema simétrico en $\xi_1$ = 0 y $\xi_2$ = a.
9. Flecha generada por un momento flector Mx(y) distribuido en línea x = xi
10. Flecha generada por un momento flector My(x) distribuido en línea y = eta

<br>

**1. Librerías necesarias**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

<br>

**2. Datos de entrada** (Unidades en el sistema internacional)

In [ ]:
# Datos de la placa:
a = ...
b = ...
h = ...
rho = ...
nu = ...
Eplaca = ...
I = h**3/12
D = Eplaca*I/(1-nu**2)

# Carga uniforme:
p = ... #Cuidado, si la carga es hacia abajo hay que indicar signo negativo

# Valores de discretización espacial
dx = 2e-2
dy = 2e-2

# Datos del pilar biarticulado
Lc = ... # longitud del pilar biarticulado
Ec = ... # módulo elástico del material del pilar biarticulado
Ac = ... # área de la sección transversal del pilar biarticulado

# Recuerde que en Python, el primer elemento de un vector tiene indice 0. Igual en una matriz, donde su primer elemento es el (0,0)
# Recuerda que puedes usar la función print() para ver el valor de alguna variable.
# Puedes también usar la función np.shape() para ver el tamaño de matrices, o alternativamente la funcion np.size("matrix",0), 0 para la dimensión filas y 1 para la dimensión 1.
# La funcion len() para saber la longitud de un vector.

Discretización del dominio espacial de la placa

In [ ]:
# Creamos dos vectores x e y, que contienen los valores discretos del espacio donde se evaluaran los cálculos
x = np.arange(0, a+dx, dx) 
y = np.arange(0, b+dy, dy)

# Creamos dos matrices X e Y, de forma que seleccionando el mismo elemento en cada una de ellas obtendremos la posición x,y de un punto cualquiera.
X, Y = np.meshgrid(x, y)
# Hacemos la traspuesta para que en las filas queden las coordenadas x, y en las columnas las coordenadas y.
X = np.transpose(X)
Y = np.transpose(Y)

Declaración de terminos de los sumatorios

In [ ]:
# Creamos dos vectores de enteros n y m, que contienen los valores maximos de los términos de los sumatorios
nmax = ... # número máximo de términos n a evaluar (más términos implica más coste computacional)
mmax = nmax
n = np.arange(1,nmax+1,1) #vector de 1 a nmax con incrementos de 1
m = np.arange(1,mmax+1,1) #vector de 1 a mmax con incrementos de 1

# Creamos dos matrices N y M, de forma que seleccionando un mismo elemento en ambas tenemos una dupla n,m
N, M = np.meshgrid(n, m)

# Hacemos la traspuesta para que en las filas sean n y las columnas m.
N = np.transpose(N)
M = np.transpose(M)

Creamos una matriz con valores Fnm que podremos usar para todos los casos de carga

In [ ]:
Fnm = ((N/a)**2 + (M/b)**2)**2

<br>

**3. Flecha y giros generados por una carga uniforme distribuida w<sub>p</sub>(x,y)** (aquí denominada w_xy_p)

In [ ]:
## Caso de carga constante uniformemente distribuida p

# Flecha w(x,y)_p
w_nm_p = 4*p/(np.pi**6*D*Fnm*N*M)*(1-(-1)**N)*(1-(-1)**M)
w_xy_p = np.zeros((len(x),len(y))) 
for i in range(len(n)):
    for j in range(len(m)):
        w_xy_p += w_nm_p[i,j]*np.sin(n[i]*np.pi*X/a)*np.sin(m[j]*np.pi*Y/b)  
            
# Giros respecto x e y, respectivamente: theta(x,y)_x_p y theta(x,y)_y_p (descomenta si necesario)
theta_y_nm_p = w_nm_p*M*np.pi/b
theta_x_nm_p = w_nm_p*N*np.pi/a
#theta_y_xy_p = np.zeros((len(x),len(y)))
#theta_x_xy_p = np.zeros((len(x),len(y)))
#for i in range(len(n)):
#    for j in range(len(m)):
#        theta_y_xy_p += theta_y_nm_p[i,j]*np.sin(n[i]*np.pi*X/a)*np.cos(m[j]*np.pi*Y/b)
#        theta_x_xy_p += theta_x_nm_p[i,j]*np.cos(n[i]*np.pi*X/a)*np.sin(m[j]*np.pi*Y/b)        

## Representemos la flecha calculada
imag_p = plt.imshow(w_xy_p*1e3, cmap='viridis', interpolation='nearest', extent=[y.min(), y.max(), x.min(), x.max()])
plt.suptitle('w(x,y)_p', x=0.57, y=0.95)
plt.ylabel('Dimension x') # Nota que los ejes están invertidos para acomodarse a la forma de representar la placa
plt.xlabel('Dimension y')
plt.gca().invert_yaxis()
plt.colorbar(imag_p,label='Flecha (mm)')
plt.show()

## Evalua la funcion w(x,y)_p en un punto de coordenadas (x_eval,y_eval). Introduce el valor en x_val e y_val
x_eval = ...
y_eval = ...
x_eval_idx = np.argmin(np.abs(x-x_eval))
y_eval_idx = np.argmin(np.abs(y-y_eval))
w_xy_p_eval = w_xy_p[x_eval_idx,y_eval_idx]
print('El valor de la flecha en el punto evaluado', '(', x_eval, ',', y_eval, ')', 'es:', "{:e}".format(w_xy_p_eval), 'm')

## Encuentra el valor de la flecha máxima en valores absolutos
w_xy_p_max = np.abs(w_xy_p).max()
print('El valor de la flecha máxima en valor absoluto es:', "{:e}".format(w_xy_p_max), 'm')
w_xy_p_max_idx = np.abs(w_xy_p).argmax()
w_xy_p_max_x_idx,w_xy_p_max_y_idx = np.unravel_index(w_xy_p_max_idx,w_xy_p.shape)
w_xy_p_max_x = w_xy_p_max_x_idx*dx
w_xy_p_max_y = w_xy_p_max_y_idx*dy
print('Las coordenadas (x,y) donde se da el máximo son:', '(', w_xy_p_max_x, ',', w_xy_p_max_y, ') m')

<br>

**4. Flecha generada por una carga puntual de valor 1 aplicada en un punto ($\xi$,$\eta$), es decir, la Función de Green del problema placa K(x,y;$\xi$,$\eta$)** 
(aquí denominada w_xy_K)

In [ ]:
## Caso de carga puntual de valor 1 aplicada en un punto (xi,eta), es decir, la función de Green del problema placa:

# Flecha w(x,y)_K
xi = ...
eta = ...
w_nm_K = 4*1*np.sin(N*np.pi*xi/a)*np.sin(M*np.pi*eta/b)/(np.pi**4*D*Fnm*a*b)
w_xy_K = np.zeros((len(x),len(y)))
for i in range(len(n)):
    for j in range(len(m)):
        w_xy_K += w_nm_K[i,j]*np.sin(n[i]*np.pi*X/a)*np.sin(m[j]*np.pi*Y/b)
        
## Evalua la funcion w(x,y)_K en un punto de coordenadas (x_eval,y_eval)
x_eval = ...
y_eval = ...
x_eval_idx = np.argmin(np.abs(x-x_eval))
y_eval_idx = np.argmin(np.abs(y-y_eval))
w_xy_K_eval = w_xy_K[x_eval_idx,y_eval_idx]
print('El valor de la flecha en el punto evaluado', '(', x_eval, ',', y_eval, ')', 'es:', "{:e}".format(w_xy_K_eval), 'm')

<br>

**5. Cálculo de fuerza reactiva F de un pilar biarticulado bajo el punto central de la placa.**

In [ ]:
F_pilar = ...
print('La fuerza reactiva F en el punto de apoyo de la placa con el pilar biarticulado, es:', "{:e}".format(F_pilar), 'N')

<br>

**6. Flecha generada por una carga puntual de valor F aplicada en un punto ($\xi$,$\eta$)** (aquí denominada w_xy_F)

In [ ]:
## Caso de carga puntual de valor F aplicada en un punto (xi,eta):

# Flecha w(x,y)_F
xi = ...
eta = ...
F = F_pilar
w_nm_F = 4*F*np.sin(N*np.pi*xi/a)*np.sin(M*np.pi*eta/b)/(np.pi**4*D*Fnm*a*b)
w_xy_F = np.zeros((len(x),len(y)))
for i in range(len(n)):
    for j in range(len(m)):
        w_xy_F += w_nm_F[i,j]*np.sin(n[i]*np.pi*X/a)*np.sin(m[j]*np.pi*Y/b)

## Representemos la flecha calculada
imag_F = plt.imshow(w_xy_F*1e3, cmap='viridis', interpolation='nearest', extent=[y.min(), y.max(), x.min(), x.max()])
plt.suptitle('w(x,y)_F', x=0.57, y=0.95)
plt.ylabel('Dimension x') # Nota que los ejes están invertidos para acomodarse a la forma de representar la placa
plt.xlabel('Dimension y')
plt.gca().invert_yaxis()
plt.colorbar(imag_F,label='Flecha (mm)')
plt.show()

## Evalua la funcion w(x,y)_F en un punto de coordenadas (x_eval,y_eval). Introduce el valor en x_val e y_val
x_eval = ...
y_eval = ...
x_eval_idx = np.argmin(np.abs(x-x_eval))
y_eval_idx = np.argmin(np.abs(y-y_eval))
w_xy_F_eval = w_xy_F[x_eval_idx,y_eval_idx]
print('El valor de la flecha en el punto evaluado', '(', x_eval, ',', y_eval, ')', 'es:', "{:e}".format(w_xy_F_eval), 'm')

## Encuentra el valor de la flecha máxima en valores absolutos
w_xy_F_max = np.abs(w_xy_F).max()
print('El valor de la flecha máxima en valor absoluto es:', "{:e}".format(w_xy_F_max), 'm')
w_xy_F_max_idx = np.abs(w_xy_F).argmax()
w_xy_F_max_x_idx,w_xy_F_max_y_idx = np.unravel_index(w_xy_F_max_idx,w_xy_F.shape)
w_xy_F_max_x = w_xy_F_max_x_idx*dx
w_xy_F_max_y = w_xy_F_max_y_idx*dy
print('Las coordenadas (x,y) donde se da el máximo son:', '(', w_xy_F_max_x, ',', w_xy_F_max_y, ') m')

<br>

**7. Cálculo de momento flector My(x) en empotramientos de problema simétrico en y = 0 e y = b.**

In [ ]:
# Coeficientes Mn del desarrollo en serie de un momento flector My(x)
eta_1 = 0
eta_2 = b
w_nm_My_x_eta_1_aux = 2*M*np.cos(M*np.pi*eta_1/b)/(b**2*np.pi**3*D*Fnm)
w_nm_My_x_eta_2_aux = 2*M*np.cos(M*np.pi*eta_2/b)/(b**2*np.pi**3*D*Fnm)
theta_y_nm_My_x_eta_1_aux = w_nm_My_x_eta_1_aux*M*np.pi/b
theta_y_nm_My_x_eta_2_aux = w_nm_My_x_eta_2_aux*M*np.pi/b

# Introduce aquí la suma de los theta_y_nm de los casos de carga que actuan simultáneamente con los momentos empotramiento y que no son incognita
theta_y_nm_input = ... # por ejemplo: theta_y_nm_K (este theta_y_nm puede no ser el correcto para tu caso)

# Cálculo de los términos del desarrollo en serie de Fourir del momento flector My(x) distribuido en el empotramiento
Mn = -np.sum(theta_y_nm_input,axis=1)/(np.sum(theta_y_nm_My_x_eta_1_aux,axis=1) - np.sum(theta_y_nm_My_x_eta_2_aux,axis=1))

# Desarrollo en serie del momento flector My(x) en el empotramiento
My_x_eta_1 = np.zeros(len(x))
My_x_eta_2 = np.zeros(len(x))
for i in range(len(n)):
    My_x_eta_1 += Mn[i]*np.sin(n[i]*np.pi*x/a)
    My_x_eta_2 -= Mn[i]*np.sin(n[i]*np.pi*x/a)

# Grafica el momento My(x) distribuido en los empotramientos
plt.plot(x, My_x_eta_1/1e3, label='My(x) en y = 0')
plt.plot(x, My_x_eta_2/1e3, label='My(x) en y = b')
plt.xlabel('Dimension x')
plt.ylabel('My(x) [kNm)]')
plt.title('Momento flector My(x) en los bordes empotrados')
plt.legend()
plt.show()

<br>

**8. Cálculo de momento flector Mx(y) en empotramientos de problema simétrico en x = 0 e x = a.**

In [ ]:
# Coeficientes Mm del desarrollo en serie de un momento flector Mx(y)
xi_1 = 0
xi_2 = a
w_nm_Mx_y_xi_1_aux = 2*N*np.cos(N*np.pi*xi_1/a)/(a**2*np.pi**3*D*Fnm)
w_nm_Mx_y_xi_2_aux = 2*N*np.cos(N*np.pi*xi_2/a)/(a**2*np.pi**3*D*Fnm)
theta_x_nm_Mx_y_xi_1_aux = w_nm_Mx_y_xi_1_aux*N*np.pi/a
theta_x_nm_Mx_y_xi_2_aux = w_nm_Mx_y_xi_2_aux*N*np.pi/a

# Introduce aquí la suma de los theta_x_nm de los casos de carga que actuan simultáneamente con los momentos empotramiento y que no son incognita
theta_x_nm_input = ... # por ejemplo: theta_x_nm_K (este theta_x_nm puede no ser el correcto para tu caso)

# Cálculo de los términos del desarrollo en serie de Fourir del momento flector Mx(y) distribuido en el empotramiento
Mm = -np.sum(theta_x_nm_input,axis=0)/(np.sum(theta_x_nm_Mx_y_xi_1_aux,axis=0) - np.sum(theta_x_nm_Mx_y_xi_2_aux,axis=0))

# Desarrollo en serie del momento flector Mx(y) en el empotramiento
Mx_y_xi_1 = np.zeros(len(y))
Mx_y_xi_2 = np.zeros(len(y))
for j in range(len(m)):
    Mx_y_xi_1 += Mm[j]*np.sin(m[j]*np.pi*y/b)
    Mx_y_xi_2 -= Mm[j]*np.sin(m[j]*np.pi*y/b)
    
# Grafica el momento My(x) distribuido en los empotramientos
plt.plot(y, Mx_y_xi_1/1e3, label='Mx(y) en x = 0')
plt.plot(y, Mx_y_xi_2/1e3, label='Mx(y) en x = a')
plt.xlabel('Dimension y')
plt.ylabel('Mx(y) [kNm)]')
plt.title('Momento flector Mx(y) en los bordes empotrados')
plt.legend()
plt.show()

<br>

**9. Flecha generada por momentos flectores Mx(y) en sendos lados empotrados x = 0 y x = a** 
Nota: puedes graficar por separado la flecha que generan los momentos flectores de los empotramientos por separado para entender mejor la placa biempotrada. La celda del cálculo no tiene código para encontrar el valor de la flecha máxima y donde se da, pero puedes copiarlo y modificarlo de otra celda que si lo tenga, por ejemplo, del caso de una carga constante uniforme w(x,y)_p

In [ ]:
#Flecha debido a Mx(y) en línea x = xi_1: w(x,y)_Mx_y_xi_1
xi_1 = 0
w_nm_Mx_y_xi_1 = 2*N*np.tile(Mm,(len(n),1))*np.cos(N*np.pi*xi_1/a)/(a**2*np.pi**3*D*Fnm)
w_xy_Mx_y_xi_1 = np.zeros((len(x), len(y)))

#Flecha debido a Mx(y) en línea x = xi_2: w(x,y)_Mx_y_xi_2
xi_2 = a
w_nm_Mx_y_xi_2 = 2*N*np.tile(-Mm,(len(n),1))*np.cos(N*np.pi*xi_2/a)/(a**2*np.pi**3*D*Fnm)
w_xy_Mx_y_xi_2 = np.zeros((len(x), len(y)))
for i in range(len(n)):
    for j in range(len(m)):
        w_xy_Mx_y_xi_1 += w_nm_Mx_y_xi_1[i,j]*np.sin(n[i]*np.pi*X/a)*np.sin(m[j]*np.pi*Y/b)
        w_xy_Mx_y_xi_2 += w_nm_Mx_y_xi_2[i,j]*np.sin(n[i]*np.pi*X/a)*np.sin(m[j]*np.pi*Y/b)
            
#Flecha total debida a los dos momentos Mx(y) actuando en sendos bordes empotrados al mismo tiempo:
w_xy_Mx_y_emp = w_xy_Mx_y_xi_1 + w_xy_Mx_y_xi_2

## Representemos la flecha calculada
imag_Mx_y_emp = plt.imshow(w_xy_Mx_y_emp*1e3, cmap='viridis', interpolation='nearest', extent=[y.min(), y.max(), x.min(), x.max()])
plt.suptitle('w(x,y)_Mx_y_emp', x=0.57, y=0.95)
plt.ylabel('Dimension x') # Nota que los ejes están invertidos para acomodarse a la forma de representar la placa
plt.xlabel('Dimension y')
plt.gca().invert_yaxis()
plt.colorbar(imag_Mx_y_emp,label='Flecha (mm)')
plt.show()

<br>

**10. Flecha generada por momentos flectores My(x) en sendos lados empotrados y = 0 e y = b** 
Nota: puedes graficar por separado la flecha que generan los momentos flectores de los empotramientos por separado para entender mejor la placa biempotrada. La celda del cálculo no tiene código para encontrar el valor de la flecha máxima y donde se da, pero puedes copiarlo y modificarlo de otra celda que si lo tenga, por ejemplo, del caso de una carga constante uniforme w(x,y)_p

In [ ]:
#Flecha debido a My(x) en línea y = eta_1: w(x,y)_My_x_eta_1
eta_1 = 0
Mn_aux = np.array(Mn).reshape(len(m),1)
w_nm_My_x_eta_1 = 2*M*np.tile(Mn_aux,(1,len(m)))*np.cos(M*np.pi*eta_1/b)/(b**2*np.pi**3*D*Fnm)
w_xy_My_x_eta_1 = np.zeros((len(x), len(y)))

#Flecha debido a My(x) en línea y = eta_2: w(x,y)_My_x_eta_2
eta_2 = b
w_nm_My_x_eta_2 = 2*M*np.tile(-Mn_aux,(1,len(m)))*np.cos(M*np.pi*eta_2/b)/(b**2*np.pi**3*D*Fnm)
w_xy_My_x_eta_2 = np.zeros((len(x), len(y)))

for i in range(len(n)):
    for j in range(len(m)):
        w_xy_My_x_eta_1 += w_nm_My_x_eta_1[i,j]*np.sin(n[i]*np.pi*X/a)*np.sin(m[j]*np.pi*Y/b)
        w_xy_My_x_eta_2 += w_nm_My_x_eta_2[i,j]*np.sin(n[i]*np.pi*X/a)*np.sin(m[j]*np.pi*Y/b)
        
#Flecha total debida a los dos momentos Mx(y) actuando en sendos bordes empotrados al mismo tiempo:
w_xy_My_x_emp = w_xy_My_x_eta_1 + w_xy_My_x_eta_2

## Representemos la flecha calculada
imag_My_x_emp = plt.imshow(w_xy_My_x_emp*1e3, cmap='viridis', interpolation='nearest', extent=[y.min(), y.max(), x.min(), x.max()])
plt.suptitle('w(x,y)_My_x_emp', x=0.57, y=0.95)
plt.ylabel('Dimension x') # Nota que los ejes están invertidos para acomodarse a la forma de representar la placa
plt.xlabel('Dimension y')
plt.gca().invert_yaxis()
plt.colorbar(imag_My_x_emp,label='Flecha (mm)')
plt.show()
